<a href="https://colab.research.google.com/github/radonys/Dog-Breed-Classification/blob/master/dog_breed_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorboardX
!pip install http://download.pytorch.org/whl/cu80/torch-0.4.1-cp36-cp36m-linux_x86_64.whl
!pip install torchvision
!pip install numpy
!pip install matplotlib
!pip install kaggle

    100% |████████████████████████████████| 71kB 4.0MB/s 
    100% |████████████████████████████████| 483.0MB 1.2MB/s 
tcmalloc: large alloc 1073750016 bytes == 0x56a50000 @  0x7f1c4611a2a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070
    100% |████████████████████████████████| 61kB 3.4MB/s 
    100% |████████████████████████████████| 2.0MB 11.2MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0
    100% |████████████████████████████████| 61kB 3.9MB/s 
    100% |████████████████████████████████| 235kB 7.2MB/s 
  Running setup.py bdist_wheel for kaggle ... - \ done
  Stored in directory: /root/.cache/pip/wheels/8b/21/3b/a0076243c6ae12a6215b2da515fe06b539aee7217b406e510e
  Running setup.py b

In [0]:
from __future__ import print_function 
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from tensorboardX import SummaryWriter
import csv
import zipfile
import shutil

In [3]:
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  0.4.1
Torchvision Version:  0.2.1


In [4]:
!mkdir -p ~/.kaggle
from google.colab import files
files.upload()
!cp kaggle.json ~/.kaggle/
!kaggle competitions download -c dog-breed-identification
!mkdir all/
!mv *.zip all/
!ls all/
!unzip all/labels.csv 
!unzip all/sample_submission.csv
!mv *.csv all/
!rm all/labels.csv.zip all/sample_submission.csv.zip
!rm kaggle.json

Saving kaggle.json to kaggle.json
  0% 0.00/214k [00:00<?, ?B/s]
100% 214k/214k [00:00<00:00, 60.5MB/s]
  0% 0.00/318k [00:00<?, ?B/s]
100% 318k/318k [00:00<00:00, 82.7MB/s]
 97% 337M/346M [00:02<00:00, 139MB/s]
100% 346M/346M [00:02<00:00, 141MB/s]
 96% 332M/345M [00:03<00:00, 135MB/s]
100% 345M/345M [00:03<00:00, 109MB/s]
labels.csv.zip	sample_submission.csv.zip  test.zip  train.zip
Archive:  all/labels.csv.zip
  inflating: labels.csv              
Archive:  all/sample_submission.csv.zip
  inflating: sample_submission.csv   


In [5]:
if os.path.exists('all/'):

	with open('all/labels.csv', mode='r') as csv_file:

		csv_reader = csv.DictReader(csv_file)
		line_count = 0
		directory = 'all/train_images/'

		if not os.path.exists(directory):
			os.makedirs(directory)


		for row in csv_reader:

			if line_count == 0:

				print('Column names are', {", ".join(row)})
				line_count += 1

			id_value = str(row["id"])
			breed = str(row["breed"])

			if not os.path.exists(os.path.join(directory, breed)):
				os.makedirs(os.path.join(directory, breed))

			traindir = 'all/train/'

			if not os.path.exists(traindir):

				zip_ref = zipfile.ZipFile('all/train.zip', 'r')
				zip_ref.extractall('all/')
				zip_ref.close()

			prev_dir = str(os.path.join(traindir, id_value + '.jpg'))
			new_dir = str(os.path.join(directory, breed, id_value + '.jpg'))
				
			if os.path.exists(os.path.join(traindir, id_value + '.jpg')):
				shutil.move(prev_dir, new_dir)

			line_count +=1

		print('Processed lines: ', line_count)
		os.rmdir(traindir)
		os.rename('all/train_images', 'all/train')

else:
	print('Data directory not found.')
  
zip_ref = zipfile.ZipFile('all/test.zip', 'r')
zip_ref.extractall('all/')
zip_ref.close()

Column names are {'id, breed'}
Processed lines:  10223


In [0]:
train = True
evaluate = False

In [0]:
writer = SummaryWriter()
data_dir = "all/"
model_name = "resnet"
num_classes = 120
batch_size = 8
num_epochs = 25
feature_extract = False

In [0]:
def save_model(model,filename):

    state = model.state_dict()
    for key in state: state[key] = state[key].clone().cpu()

    if not os.path.exists('all/saved_models/'):
        os.makedirs('all/saved_models')

    torch.save(state, os.path.join('all/saved_models',filename))

In [0]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):

    since = time.time()

    train_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):

        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train']:
            
            if phase == 'train':
                model.train()  # Set model to training mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:

                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            writer.add_scalar('data/Training_Accuracy', 100.0*epoch_acc, epoch)
        	writer.add_scalar('data/Training_Loss', epoch_loss, epoch)

            # deep copy the model
            if epoch_acc > best_acc:

                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                save_model(model, '{}_{}.pth'.format('dog-breed-resnet18',epoch))

            train_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best Training Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, train_acc_history

In [0]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [0]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    else:
        print("Invalid model name, exiting...")
        exit()
    
    return model_ft, input_size

In [0]:
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
print(model_ft)

In [0]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train']}
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train']}

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = model_ft.to(device)

NameError: ignored

In [0]:
params_to_update = model_ft.parameters()
print("Params to learn:")

if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

In [0]:
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

In [0]:
#Load saved model, if any
#model_ft.load_state_dict(torch.load('all/saved_models/...'))

In [0]:
criterion = nn.CrossEntropyLoss()

In [0]:
if train:

    model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs)

    ohist = []
    ohist = [h.cpu().numpy() for h in hist]
    plt.title("Training Accuracy vs. Number of Training Epochs")
    plt.xlabel("Training Epochs")
    plt.ylabel("Training Accuracy")
    plt.plot(range(1,num_epochs+1),ohist,label="Pretrained over ImageNet")
    plt.ylim((0,1.))
    plt.xticks(np.arange(1, num_epochs+1, 1.0))
    plt.legend()
    plt.savefig('train-vs-epoch.png')